In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.io import savemat
import warnings

warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from connectomics.atlases import fetch_dk_atlas, get_dk_hemisphere_mask
from connectomics.connectivity import merge_connectomes
from utils.transform import minmax_scale_data

# in
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

df_dir = os.path.join(data_dir, 'datasets')
masks_dir = os.path.join(data_dir, 'masks')

df_fname = os.path.join(df_dir, 'bf2_tau_asymmetry_ad_t_xs_tnic_fnc_mri.csv')

# out
nbs_in_dir = os.path.join(data_dir, 'nbs', 'in')
os.makedirs(nbs_in_dir, exist_ok=True)
nbs_out_dir = os.path.join(data_dir, 'nbs', 'out')
os.makedirs(nbs_out_dir, exist_ok=True)

In [2]:
# load Desikan-Killiany atlas
dk_img, dk_labels, dk_coords = fetch_dk_atlas(coords=True)

# save atlas coordinates and labels
np.savetxt(os.path.join(nbs_in_dir, 'dk_coords.txt'), dk_coords)
with open(os.path.join(nbs_in_dir, 'dk_labels.txt'), 'w') as f:
        for label in list(dk_labels.keys()):
                f.write(f"{label}\n")

In [8]:
mask_perc = 90
modality = 'fc' # 'sc'
corr_type = f'dk_{modality}_file'
mask_type = f'dk_n_{modality}_mask'
grp_col = 'temporal_meta_tau_asymmetry_group'

In [9]:
# load normative masks
no_mask = get_dk_hemisphere_mask(dk_labels, 'full') #np.ones((len(dk_labels), len(dk_labels)))
interhemi_mask = get_dk_hemisphere_mask(dk_labels, 'interhemi') #np.ones((len(dk_labels), len(dk_labels)))
hc_normative_mask = np.load(os.path.join(masks_dir, f'{mask_type}_{mask_perc}.npy'))

# Read the datasets for AD and HC groups
df_ad = pd.read_csv(df_fname, index_col=0, low_memory=False)

# leave only ones with MRI
df_ad = df_ad[~df_ad[corr_type].isna()]

# Create dataframes for each of the asymmetry groups
df_A = df_ad[df_ad[grp_col].isin(['LA', 'RA'])].reset_index(drop=True)
df_LA = df_ad[df_ad[grp_col]=='LA'].reset_index(drop=True)
df_S = df_ad[df_ad[grp_col]=='S'].reset_index(drop=True)
df_RA = df_ad[df_ad[grp_col]=='RA'].reset_index(drop=True)

all_masks = {
    f'no_mask' : no_mask,
    f'normal_mask_{mask_perc}' : hc_normative_mask,
}

if corr_type == 'dk_fc_file': 
    drop_first_roi = True # labels in FC matrix to be removed
    fisher = True # apply fisher r-to-z
    replace_nan_with = None
elif corr_type == 'dk_sc_file': 
    drop_first_roi = False
    fisher = False
    replace_nan_with = None
elif corr_type == 'dk_fa_file' or corr_type == 'dk_md_file':
    drop_first_roi = False
    fisher = False
    replace_nan_with = 0

conn_params = dict(connectome_path_col=corr_type, drop_first_roi=drop_first_roi, 
                   fisher=fisher, replace_nan_with=replace_nan_with, verbose=True)

for mask_name, mask in all_masks.items():
    print(f"Applying and exporting correlation matrices with {mask_name}")
    df_A, connectomes_A = merge_connectomes(df_A, mask=mask, **conn_params)
    df_LA, connectomes_LA = merge_connectomes(df_LA, mask=mask, **conn_params)
    df_S, connectomes_S = merge_connectomes(df_S, mask=mask, **conn_params)
    df_RA, connectomes_RA = merge_connectomes(df_RA, mask=mask, **conn_params)

    savemat(os.path.join(nbs_in_dir, f'{corr_type}_A_{mask_name}.mat'), 
            {f'{corr_type}_A': connectomes_A})
    savemat(os.path.join(nbs_in_dir, f'{corr_type}_LA_{mask_name}.mat'), 
            {f'{corr_type}_LA': connectomes_LA})
    savemat(os.path.join(nbs_in_dir, f'{corr_type}_S_{mask_name}.mat'), 
            {f'{corr_type}_S': connectomes_S})
    savemat(os.path.join(nbs_in_dir, f'{corr_type}_RA_{mask_name}.mat'), 
            {f'{corr_type}_RA': connectomes_RA})

print(f"Data shape (A): {df_A.shape}")
print(f"Data shape (LA): {df_LA.shape}")
print(f"Data shape (S): {df_S.shape}")
print(f"Data shape (RA): {df_RA.shape}")

Applying and exporting correlation matrices with no_mask
Applying and exporting correlation matrices with normal_mask_90
Data shape (A): (103, 3195)
Data shape (LA): (73, 3195)
Data shape (S): (215, 3195)
Data shape (RA): (30, 3195)


In [10]:
tau_col = 'tnic_global'

# Create a subset of design matrix of covariates
df_design_A = df_A[['age', 'gender_baseline_variable', tau_col]].copy()
df_design_A[['age', tau_col]] = minmax_scale_data(df_design_A, ['age', tau_col])
design_mat_A = df_design_A.to_numpy()
savemat(os.path.join(nbs_in_dir, f'covars_{modality}_A.mat'), 
        {f'covars_A': design_mat_A})

df_design_LA = df_LA[['age', 'gender_baseline_variable', tau_col]].copy()
df_design_LA[['age', tau_col]] = minmax_scale_data(df_design_LA, ['age', tau_col])
design_mat_LA = df_design_LA.to_numpy()
savemat(os.path.join(nbs_in_dir, f'covars_{modality}_LA.mat'), 
        {f'covars_LA': design_mat_LA})

df_design_RA = df_RA[['age', 'gender_baseline_variable', tau_col]].copy()
df_design_RA[['age', tau_col]] = minmax_scale_data(df_design_RA, ['age', tau_col])
design_mat_RA = df_design_RA.to_numpy()
savemat(os.path.join(nbs_in_dir, f'covars_{modality}_RA.mat'), 
        {f'covars_RA': design_mat_RA})

df_design_S = df_S[['age', 'gender_baseline_variable', tau_col]].copy()
df_design_S[['age', tau_col]] = minmax_scale_data(df_design_S, ['age', tau_col])
design_mat_S = df_design_S.to_numpy()
savemat(os.path.join(nbs_in_dir, f'covars_{modality}_S.mat'), 
        {f'covars_S': design_mat_S})